### 1. Deteksi pengambilan data wajah 
### 2. Training Dataset dan simpan di file Training.yml
### 3. Pengenalan wajah

# 1. Pengambilan data wajah

In [ ]:
import cv2, os 
import numpy as np
from tkinter import *

#Mengambil sampel data wajah yang akan di masukkan kedalam folder dataset
wajahDir = 'dataset'
cam = cv2.VideoCapture(0)
cam.set(3, 640) #ubah lebar cam
cam.set(4, 480) #ubah tinggi cam

#deteksi wajah dan mata menggunakan file haarcascade_frontalface_default.xml
faceDetector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyeDetector = cv2.CascadeClassifier('haarcascade_eye.xml')

#rekam data wajah dengan menggunakan ID 
facesID = input("Masukkan ID untuk rekam data [kemudian tekan ENTER] : ")
print("Posisikan wajah anda di depan webcam. Tunggu proses pengambilan wajah selesai..")
ambilData = 1
while True:
    retV, frame = cam.read()
    frame = cv2.flip(frame, 1) #fertical flip
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #threshold_value = 128     # Menentukan threshold
    #threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1) #ubh gmbr jdi threshold
    faces = faceDetector.detectMultiScale(frame, 1.3, 5) #frame, scaleFctor, minNeighbors
    for (x, y, w, h) in faces:
        frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
        namaFile = "Wajah."+str(facesID)+'.'+str(ambilData)+'.jpg'
        cv2.imwrite(wajahDir+'/'+namaFile,gray)
        ambilData += 1
        roiGray = gray[y:y+h, x:x+w]
        roiWarna = frame[y:y+h, x:x+w]
        eyes = eyeDetector.detectMultiScale(roiGray)
        for (xe,ye,we,he) in eyes:
            cv2.rectangle(roiWarna,(xe,ye),(xe+we,ye+he),(0,0,255),1)
    cv2.imshow("Kamera Nyala",frame)
    #cv2.imshow("Kamera gray Nyala", gray)
    k = cv2.waitKey(1) & 0xff
    if k == 27 or k == ord("x"):
    #if cv2.waitKey(1) & 0xff == ord("x"):
        break
    elif ambilData>40:
        break
print("Pengambilan data selesai.")
cam.release()
cv2.destroyAllWindows()

# 2. Training Dataset

In [ ]:
#Latihan wajah dengan menggunakan metode LBPH
import cv2, os
import numpy as np
from PIL import Image

wajahDir = 'dataset'
latihDir = 'trainingdata'
def getImageLabel(path):
    ImagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples = []
    faceIDs = []
    for imagePath in ImagePaths:
        PILImg = Image.open(imagePath).convert('L') #convert kedalam grey
        imgNum = np.array(PILImg,'uint8')
        faceID = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = faceDetector.detectMultiScale(imgNum)
        
        for (x, y, w, h) in faces:
            faceSamples.append(imgNum[y:y+h,x:x+w])
            faceIDs.append(faceID)
        return faceSamples,faceIDs

#Menggunakan metode Local Binary Pattern Histogram
faceRecognizer = cv2.face.LBPHFaceRecognizer_create()
faceDetector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
print("Sedang melakukan training data wajah. Tunggu dalam beberapa detik...")
faces, IDs = getImageLabel(wajahDir)
faceRecognizer.train(faces,np.array(IDs))

#Simpan file Training ke dalam folder trainingdata
faceRecognizer.write(latihDir+'/training.yml')
print ("Data wajah telah di Trainingkan..".format(len(np.unique(IDs))))


# 3. Face Recognition

In [ ]:
import cv2, os
import numpy as np
import datetime
import mysql.connector
from tkinter import *


db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="1234",
    database="recognizer"
)



wajahDir = 'dataset'
latihDir = 'trainingdata'
cam = cv2.VideoCapture(0)
cam.set(3, 640) #ubah lebar cam
cam.set(4, 480) #ubah tinggi cam

#deteksi wajah dan mata menggunakan file haarcascade_frontalface_default.xml
faceDetector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faceRecognizer = cv2.face.LBPHFaceRecognizer_create()

faceRecognizer.read(latihDir+'/training.yml')
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
names = ['Jalaluddin',' Tidak Diketahui ', ' ']

minWidth = 0.1*cam.get(3)
minHeight = 0.1*cam.get(4)

while True:
    retV, frame = cam.read()
    frame = cv2.flip(frame, 1) #fertical flip
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #threshold_value = 128     # Menentukan threshold
    #threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)
    faces = faceDetector.detectMultiScale(gray, 1.2, 5, minSize=(round(minWidth),round(minHeight)),) #frame, scaleFactor, minNeighbors
    for (x, y, w, h) in faces:
        frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        id, confidence = faceRecognizer.predict(gray[y:y+h,x:x+w]) # confidence = 0 artinya cocok sempurna
        if confidence>=50:
            nameID = names[id]
            confidenceTxt = "{0}".format(round(100-confidence))
        else:
            nameID = names[0]
            
            confidenceTxt = "{0}".format(round(100-confidence))
        cv2.putText(frame, str(nameID),(x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(frame, str(confidenceTxt),(x+5,y+h-5), font, 1, (0,255,0), 1)

    cv2.imshow("Mengenali Wajah",frame)
    #cv2.imshow("Kamera gray Nyala", gray)
    #cv2.imshow("Kamera threshold", threshold)

    k = cv2.waitKey(1) & 0xff
    if k == 27 or k == ord("x"):
    #if cv2.waitKey(1) & 0xff == ord("x"):
        break
 
print("Keluar")
cam.release()
cv2.destroyAllWindows()